In [8]:
import geopandas as gpd
import numpy as np
import os

# Import geomap of the Netherlands
mapdf = gpd.read_file('Data/NL-geomap/gemeente_2018_v3.shp')
mapdf = mapdf[mapdf["H2O"] == "NEE"]
mapdf.sort_values("GM_NAAM", inplace=True)
mapdf = mapdf.reset_index(drop=True)
print(mapdf.columns)

# Compute distances between all municipalities
mapdf['GEO_CENTROID'] = mapdf.centroid
centres = mapdf['GEO_CENTROID']
n_mun = mapdf.shape[0]

dist_matrix = np.zeros(shape=(n_mun, n_mun))
for i in range(0, n_mun):
    dist_matrix[i] = centres.distance(centres[i])


# Compute mobility based gravity model
# A_ij = (M_i * M_j) * (SUM_j(1 / d_ij) * n)
population = mapdf['AANT_INW']
mob_matrix = np.zeros(shape=(n_mun, n_mun))
for i in range(0, n_mun):
    sum_inverse_dist = 0
    for j in range(0, n_mun):
        if i != j:
            sum_inverse_dist += (1 / dist_matrix[i,j])
            mob_matrix[i,j] = population[i] * population[j]
    mob_matrix[i] = mob_matrix[i] * (sum_inverse_dist / n_mun)

# Normalize the mobility matrix
for i in range(0, n_mun):
    mob_matrix[i] = mob_matrix[i] / population[i]

# Save the new matrix
path = os.getcwd() + '/Data/'
np.save(path + 'Randomized_mob_grav', mob_matrix)


Index(['GM_CODE', 'GM_NAAM', 'H2O', 'OAD', 'STED', 'BEV_DICHTH', 'AANT_INW',
       'AANT_MAN', 'AANT_VROUW', 'P_00_14_JR',
       ...
       'AV20PODIUM', 'AF_MUSEUM', 'AV5_MUSEUM', 'AV10MUSEUM', 'AV20MUSEUM',
       'JRSTATCODE', 'JAAR', 'Shape_Leng', 'Shape_Area', 'geometry'],
      dtype='object', length=205)
